<a href="https://colab.research.google.com/github/COTILab/MCX24Workshop/blob/master/Training/MCX2024_2C_redbird_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!#@title Init 1: Initial setup of environment for running MCXLAB (run the below section once)
# install octave and oct2py
! sudo apt-get update && sudo apt-get install octave jq libpocl2 pocl-opencl-icd    # install octave (a free matlab clone)
! pip install oct2py jdata bjdata matplotlib   # install oct2py, jdata, bjdata and matplotlib Python modules
#!apt-get install nvidia-opencl-icd-384        # optional: install nvidia OpenCL(+2min), needed for mmc/mmclab demos later
# add octave support to colab notebook
%load_ext oct2py.ipython
# download and unzip mcxlab
! rm -rf mcxlab*
! wget https://mcx.space/nightly/release/MCX24/MCXStudio-linux-x86_64-nightlybuild.zip  # download MCX suite
! unzip -o MCXStudio-linux-x86_64-nightlybuild.zip && rm -rf MCXStudio-linux-x86_64-nightlybuild.zip # unzip MCX suite

# add executable to the shell's paths
! sudo cp -a $PWD/MCXStudio/MCXSuite/mcx/bin/mcx /usr/local/bin
! sudo cp -a $PWD/MCXStudio/MCXSuite/mcxcl/bin/mcxcl /usr/local/bin
! sudo cp -a $PWD/MCXStudio/MCXSuite/mmc/bin/mmc /usr/local/bin
# if a GPU runtime can not be allocated, run this section to setup OpenCL and MCXLAB-CL
! [[ ! -f `which nvidia-smi` ]] && sudo mv /usr/local/bin/mcxcl /usr/local/bin/mcx
! mcx -L

In [ ]:
%%octave

addpath([pwd filesep 'MCXStudio/MATLAB/redbird-m' filesep 'matlab']); % add path to redbird-m
addpath([pwd filesep 'MCXStudio/MATLAB/mcxlab']);                     % add path to mcxlab
addpath([pwd filesep 'MCXStudio/MCXSuite/mcx' filesep 'utils']);      % add path to mcx helper functions
addpath([pwd filesep 'MCXStudio/MATLAB/mmclab']);                     % add path to mmclab
addpath([pwd filesep 'MCXStudio/MCXSuite/mmc' filesep 'matlab']);     % add path to mmc helper functions
addpath([pwd filesep 'MCXStudio/MATLAB/iso2mesh']);                   % add path to iso2mesh/jsonlab
addpath([pwd filesep 'MCXStudio/MATLAB/brain2mesh']);                 % add path to brain2mesh
addpath([pwd filesep 'MCXStudio/MATLAB/zmat']);                       % add path to zmat for data compression

struct_levels_to_print(0);
print_struct_array_contents(0);

if(system('which nvidia-smi'))                   % if this runtime does not have NVIDIA GPU support
    addpath([pwd filesep 'MCXStudio/MATLAB/mcxlabcl']);    % add path to mcxlabcl
    USE_MCXCL=1;                                 % set USE_MCXCL flag in the base workspace
end

## **Take home messages:**

In this session you will learn how to:
 run and visualize frequency domain (FD) or continuous wave (CW) forward

1.   Setup the simulation environment in Redbird-m to perform forward simulations and reconstructions
2.   Run and visualize frequency domain (FD) or continuous wave (CW) forward simulations in Redbird-m using point-source/detector arrays and wide-field source/detector patterns
3.   Visualize the simulated data (amplitude and/or phase) and the sensitivity matrix


## Forward validation of Redbird-m

In this section, we compare the forward solutions produced by Redbird-m and mcx to validate that both toolboxes produce similar results.

We show you examples of the forward solutions produced on a homogeneous domain using a single point source or an array of wide-field patterns.

In general, Redbird-m and mmclab (as well as mcxlab) share a similar data structure - i.e. the simulation settings are passed onto redbird functions in the form a MATLAB/Octave `struct` object, similar to the `cfg` struct we discussed in earlier sessions.

### Single point-source forward validation
In the below example we prepare the general setting for running a forward simulation of a homogeneous domain using a single point source for both Redbird-m and MCX.

#### Define general parameters for Redbird-m single point-source forward simulation

In [ ]:
%%octave

clear cfg0
clc

% Generate homogeneous domain
domsz   = [50 50 40]; % Domain size
resmesh = 2;
[nobbx,elembbx] = meshgrid5(0:resmesh:domsz(1),0:resmesh:domsz(2),0:resmesh:domsz(3)); % Meshbox domain (in mm)
fcbbx   = volface(elembbx);
[cfg0.node, cfg0.elem] = s2m(nobbx,fcbbx(:,1:3),1,25,'tetgen1.5',[11 1 1]); % Generate measured/real heterogeneous domain
cfg0.elem = cfg0.elem(:,1:4);
clear nobbx elembbx fcbbx
cfg0.seg = ones(size(cfg0.elem,1),1);

% Visualize domain
figure, plotmesh(cfg0.node, cfg0.elem,'y>25')

% General settings for RB simulations
cfg0.prop  = [0       0 1 1   ; ...
              0.006 0.8 0 1.37];

% Define RB point source simulation parameters
cfg0.srcpos = [24.5 24.5 0];
cfg0.srcdir = [0 0  1];
cfg0.detpos = [34.5 24.5 domsz(3)];
cfg0.detdir = [0 0 -1];

cfg = cfg0;
[cfg0,sd0] = rbmeshprep(cfg0);

% Run RB forward simulation
[detphi0,phi0] = rbrunforward(cfg0,'sd',sd0);


#### Define general parameters for MCX single point-source forward simulation

In [ ]:
%%octave

clear xcfg
clc

xcfg.nphoton    = 1e8;
xcfg.vol        = uint8(ones(domsz));
xcfg.gpuid      = 1;
xcfg.autopilot  = 1;
xcfg.prop       = cfg0.prop;
xcfg.tstart     = 0;
xcfg.tend       = 5e-9;
xcfg.tstep      = 5e-9;
xcfg.seed       = 99999;
xcfg.issrcfrom0 = 0;
xcfg.isreflect  = 1;
xcfg.unitinmm   = 1;

% Define MCX point source forward simulation
xcfg.srcpos     = [25 25 0];
xcfg.srcdir     = [0 0 1];

% Run MCX forward simulation
flux = mcxlab(xcfg);
fcw  = flux.data*xcfg.tstep;


#### Compare MCX & Redbird-m solutions for single point-source solutions

In [ ]:
%%octave

clc

clines = 0:-0.5:-8;
[xi,yi] = meshgrid(0.5:domsz(1)-0.5,0.5:domsz(3)-0.5);

if isfield(cfg0,'srcpos0')
  zslice = cfg0.srcpos0(1)+1;
else
  zslice = cfg0.srcpos(1)+1;
end
clim = [-8 0];

figure,
subplot(2,2,1)
contourf(xi,yi,rot90(log10(abs(squeeze(fcw(:,xcfg.srcpos(2),:)))),-1),clines,'r-','LineWidth',2)
set(gca,'xlim',[0 domsz(1)],'ylim',[0 domsz(3)],'YDir','Normal','XDir','Reverse')
axis equal
axis tight
colorbar
caxis([clim(1) clim(2)])
title('MCX solution');

subplot(2,2,2)
plotmesh([cfg0.node full(log10(phi0(:,1)))],cfg0.elem,char(strcat('x>',num2str(zslice))))
view([-1 0 0]);
shading interp;
colorbar;
caxis([clim(1) clim(2)])
title('Redbird solution');


subplot(2,2,3:4)
clines = 0:-0.5:-8;
[xi,yi] = meshgrid(0.5:domsz(1)-0.5,0.5:domsz(3)-0.5);
[cutpos,cutvalue,facedata] = qmeshcut(cfg0.elem,cfg0.node,phi0(:,1),char(strcat('x=',num2str(zslice))));
vphi = griddata(cutpos(:,2),cutpos(:,3),cutvalue,xi+0.5,yi+0.5);

[c,h] = contour(xi,yi,log10(vphi),clines,'r-','LineWidth',2);
cwf   = squeeze(fcw(xcfg.srcpos(1),:,:,1))';
hold on,contour(xi,yi,log10(cwf),clines,'b-','LineWidth',2);
legend('RedBird','MCX')


### Wide-field source forward validation
In the below example we prepare the general setting for running a forward simulation of a homogeneous domain using a set of sliding-bar source patterns for both Redbird-m and MCX.

#### Define general parameters for Redbird-m wide-field source forward simulation

In this section we first generate a 50x50x40 mm homogeneous mesh; then we create a set of 16 sliding-bar source patterns (8 in x-direction and 8 in y-direction) and run the forward simulation in Redbirm-m.

In [ ]:
%%octave

clear cfg0
clc

% Generate homogeneous domain
domsz   = [50 50 40]; % Domain size
resmesh = 2;
[nobbx,elembbx] = meshgrid5(0:resmesh:domsz(1),0:resmesh:domsz(2),0:resmesh:domsz(3)); % Meshbox domain (in mm)
fcbbx   = volface(elembbx);
[cfg0.node, cfg0.elem] = s2m(nobbx,fcbbx(:,1:3),1,25,'tetgen1.5',[11 1 1]); % Generate measured/real heterogeneous domain
cfg0.elem = cfg0.elem(:,1:4);
clear nobbx elembbx fcbbx
cfg0.seg = ones(size(cfg0.elem,1),1);

% Visualize domain
figure, plotmesh(cfg0.node, cfg0.elem,'y>25')

% General settings for RB simulations
cfg0.prop  = [0       0 1 1   ; ...
              0.006 0.8 0 1.37];

% Define RB wide-field source simulation parameters
nBarpats   = 4; % Number of bar patterns in every x/y directions
srcpattern = diag(ones(1,nBarpats));
srcpattern = permute(repmat(srcpattern,[1,1,size(srcpattern, 1)]),[2 3 1]);
srcpattern = cat(3,srcpattern,permute(srcpattern,[2 1 3]));
detpattern = srcpattern;
cfg0.srctype    = 'pattern';
cfg0.srcpattern = srcpattern;
cfg0.srcpos     = [9.5 9.5 0];
cfg0.srcparam1  = [ 30   0 0 0];
cfg0.srcparam2  = [  0  30 0 0];
cfg0.srcdir     = [  0   0 1];
cfg0.srcweight  = ones(1,nBarpats);
cfg0.dettype    = 'pattern';
cfg0.detpattern = detpattern;
cfg0.detpos     = [9.5 9.5 domsz(3)];
cfg0.detparam1  = [ 30   0  0 0];
cfg0.detparam2  = [  0  30  0 0];
cfg0.detdir     = [  0   0 -1];
cfg0.detweight  = ones(1,nBarpats);

cfg = cfg0;
[cfg0,sd0] = rbmeshprep(cfg0);

% Run RB forward simulation
[detphi0,phi0] = rbrunforward(cfg0,'sd',sd0);

#### Define general parameters for MCX wide-field source forward simulation

In this section we first generate a 50x50x40 mm homogeneous voxelized domain; then we create a set of 16 sliding-bar source patterns (8 in x-direction and 8 in y-direction) and rn=un the forward simulation in MCX.

In [ ]:
%%octave

clear xcfg
clc

xcfg.nphoton    = 1e8;
xcfg.vol        = uint8(ones(domsz));
xcfg.gpuid      = 1;
xcfg.autopilot  = 1;
xcfg.prop       = cfg0.prop;
xcfg.tstart     = 0;
xcfg.tend       = 5e-9;
xcfg.tstep      = 5e-9;
xcfg.seed       = 99999;
xcfg.issrcfrom0 = 0;
xcfg.isreflect  = 1;
xcfg.unitinmm   = 1;

% Define MCX wide-field source forward simulation
xcfg.srctype    = 'pattern';
xcfg.srcpattern = permute(srcpattern,[3 1 2]);
xcfg.srcnum     = size(srcpattern,3);
xcfg.srcdir     = [0 0 1];
xcfg.srcpos     = [10 10 0];
xcfg.srcparam1  = [30  0 0 size(srcpattern,1)];
xcfg.srcparam2  = [ 0 30 0 size(srcpattern,2)];
xcfg.outputtype = 'fluence';

% Run MCX forward simulation
flux = mcxlab(xcfg);
fcw  = flux.data;

#### Compare MCX & Redbird-m solutions for sliding-bar source pattern solutions



In [ ]:
%%octave

clc

clines = 0:-0.5:-8;
[xi,yi] = meshgrid(0.5:domsz(1)-0.5,0.5:domsz(3)-0.5);

if isfield(cfg0,'srcpos0')
  zslice = cfg0.srcpos0(1)+1;
else
  zslice = cfg0.srcpos(1)+1;
end
clim = [-8 0];


figure,
subplot(2,2,1);
contourf(rot90(log10(abs(squeeze(fcw(xcfg.srcpos(1),:,:,1)))),-1),clines,'r-','LineWidth',2)
set(gca,'xlim',[0 domsz(1)],'ylim',[0 domsz(3)],'YDir','Normal','XDir','Reverse')
axis equal
axis tight
colorbar
caxis([clim(1) clim(2)])
title('MCX solution');

subplot(2,2,2);
plotmesh([cfg0.node full(log10(phi0(:,1)))],cfg0.elem,char(strcat('x>',num2str(zslice))))
view([-1 0 0]);
shading interp;
colorbar;
caxis([clim(1) clim(2)])
title('Redbird solution');

subplot(2,2,3:4)
clines = 0:-0.5:-8;
[xi,yi] = meshgrid(0.5:domsz(1)-0.5,0.5:domsz(3)-0.5);
[cutpos,cutvalue,facedata] = qmeshcut(cfg0.elem,cfg0.node,phi0(:,1),char(strcat('x=',num2str(zslice))));
vphi = griddata(cutpos(:,2),cutpos(:,3),cutvalue,xi+0.5,yi+0.5);

[c,h] = contour(xi,yi,log10(vphi),clines,'r-','LineWidth',2);
cwf   = squeeze(fcw(xcfg.srcpos(1),:,:,1))';
hold on,contour(xi,yi,log10(cwf),clines,'b-','LineWidth',2);
legend('RedBird','MCX')


## Getting to know Redbird-m forward functionalities

In the below sections we introduce some of the functionalities of Redbird-m. We show how to produce forward simulations of continuous (CW) and frequency-domain (FD) illuminations using either arrays of point-sources/detector or sets of wide-field source/detector patterns.

### Forward simulations of CW/FD point-source arrays using Redbird-m

In this example we generate a 50x50x40 homogeneous domain to run forward simulations of an array of 3x3 point-sources located at the bottom syrface of the mesh as well as an array of 3x3 point-detectors located at the top surface of the domain. We demonstrate the capability of Redbird-m to produce forward solution of CW or FD point illumination and detection.

In [ ]:
%%octave

clear cfg0
clc

% Generate homogeneous domain
domsz   = [50 50 40]; % Domain size
resmesh = 2;
[nobbx,elembbx] = meshgrid5(0:resmesh:domsz(1),0:resmesh:domsz(2),0:resmesh:domsz(3)); % Meshbox domain (in mm)
fcbbx   = volface(elembbx);
[cfg0.node, cfg0.elem] = s2m(nobbx,fcbbx(:,1:3),1,25,'tetgen1.5',[11 1 1]); % Generate measured/real heterogeneous domain
cfg0.elem = cfg0.elem(:,1:4);
clear nobbx elembbx fcbbx
cfg0.seg = ones(size(cfg0.elem,1),1);

% Visualize domain
figure, plotmesh(cfg0.node, cfg0.elem,'y>25')

% General settings for RB simulations
cfg0.prop  = [0       0 1 1   ; ...
              0.006 0.8 0 1.37];
mfreq      = 0 # @param [0, 135] {type:"raw"}
cfg0.omega = mfreq*1e6*2*pi;

% Define parameters for RB point-source array simulation
nsrc        = [3 3]; % Number of point src/det in xy axis
[xi,yi]     = meshgrid(linspace(10,40,nsrc(1)),linspace(10,40,nsrc(2)));
cfg0.srcpos = [xi(:),yi(:),zeros(numel(yi),1)];
cfg0.detpos = [xi(:),yi(:),domsz(3)*ones(numel(yi),1)];
cfg0.srcdir = [0 0  1];
cfg0.detdir = [0 0 -1];

cfg = cfg0;
[cfg0,sd0] = rbmeshprep(cfg0);

% Run RB forward simulation
[detphi0,phi0] = rbrunforward(cfg0,'sd',sd0);


### Visualize Redbird-m point-source array forward simulations

In [ ]:
%%octave

clc

% Select src/det optodes to visualize
idxsrc = 8 # @param [1,2,3,4,5,6,7,8,9] {type:"raw"}
detid  = 9 # @param [1,2,3,4,5,6,7,8,9] {type:"raw"}
idxdet = detid + 9;

if ~isreal(phi0) % If FD simulation
  nplot = 2;
else
  nplot = 1;
end

figure,
subplot(nplot,2,1),plotmesh([cfg0.node log(abs(phi0(:,idxsrc)))],cfg0.elem,char(strcat('y>25'))),shading interp,colorbar,title('Src Log Amplitude') % Visualize fluence amplitude from det#10
subplot(nplot,2,2),plotmesh([cfg0.node log(abs(phi0(:,idxdet)))],cfg0.elem,char(strcat('y>25'))),shading interp,colorbar,title('Det Log Amplitude') % Visualize fluence amplitude from det#10
if ~isreal(phi0)
  subplot(nplot,2,3),plotmesh([cfg0.node   angle(phi0(:,idxsrc))],cfg0.elem,char(strcat('y>25'))),shading interp,colorbar,title('Src Phase') % Visualize fluence phase from det#10
  subplot(nplot,2,4),plotmesh([cfg0.node   angle(phi0(:,idxdet))],cfg0.elem,char(strcat('y>25'))),shading interp,colorbar,title('Det Phase') % Visualize fluence phase from det#10
end

% Visualize sensitivity matrix
figure,plotmesh([cfg0.node   log(abs(prod(phi0(:,[idxsrc idxdet]),2)))],cfg0.elem,'y>25') ,shading interp,colorbar,title('Amplitude Sensitivity Matrix') % Visualize sensitivity matrix src#1 & det#10
if ~isreal(phi0)
  figure,plotmesh([cfg0.node angle(prod(phi0(:,[idxsrc idxdet]),2))],cfg0.elem,'y>25') ,shading interp,colorbar,title('Phase Sensitivity Matrix') % Visualize sensitivity matrix src#1 & det#10
end


### Forward simulations of CW/FD wide-field source/detector patterns using Redbird-m

In this example we generate a 50x50x40 homogeneous domain to run forward simulations of a total of 8 wide-field source patterns (4 in x-direction, 4 in y-direction) located at the bottom surface of the mesh as well as similar set of 8 wide-field detector patterns (4 in x-direction, 4 in y-direction) located at the top surface of the domain. We demonstrate the capability of Redbird-m to produce forward solution of CW or FD wide-field illuminations/detections.

In [ ]:
%%octave

clear cfg0
clc

% Generate homogeneous domain
domsz   = [50 50 40]; % Domain size
resmesh = 2;
[nobbx,elembbx] = meshgrid5(0:resmesh:domsz(1),0:resmesh:domsz(2),0:resmesh:domsz(3)); % Meshbox domain (in mm)
fcbbx   = volface(elembbx);
[cfg0.node, cfg0.elem] = s2m(nobbx,fcbbx(:,1:3),1,25,'tetgen1.5',[11 1 1]); % Generate measured/real heterogeneous domain
cfg0.elem = cfg0.elem(:,1:4);
clear nobbx elembbx fcbbx
cfg0.seg = ones(size(cfg0.elem,1),1);

% Visualize domain
figure, plotmesh(cfg0.node, cfg0.elem,'y>25')

% General settings for RB simulations
cfg0.prop  = [0       0 1 1   ; ...
              0.006 0.8 0 1.37];
mfreq      = 135 # @param [0, 135] {type:"raw"}
cfg0.omega = mfreq*1e6*2*pi;

% Define RB wide-field source simulation parameters
nBarpats   = 4; % Number of bar patterns in x/y directions
srcpattern = diag(ones(1,nBarpats));
srcpattern = permute(repmat(srcpattern,[1,1,size(srcpattern, 1)]),[2 3 1]);
srcpattern = cat(3,srcpattern,permute(srcpattern,[2 1 3]));
detpattern = srcpattern;
cfg0.srctype    = 'pattern';
cfg0.srcpattern = srcpattern;
cfg0.srcpos     = [10 10 0];
cfg0.srcparam1  = [30  0 0 0];
cfg0.srcparam2  = [ 0 30 0 0];
cfg0.srcdir     = [ 0  0 1];
cfg0.srcweight  = ones(1,nBarpats);
cfg0.dettype    = 'pattern';
cfg0.detpattern = detpattern;
cfg0.detpos     = [10 10 domsz(3)];
cfg0.detparam1  = [30  0  0 0];
cfg0.detparam2  = [ 0 30  0 0];
cfg0.detdir     = [ 0  0 -1];
cfg0.detweight  = ones(1,nBarpats);

cfg = cfg0;
[cfg0,sd0] = rbmeshprep(cfg0);

% Run RB forward simulation
[detphi0,phi0] = rbrunforward(cfg0,'sd',sd0);

### Visualize Redbird-m wide-field source/detector pattern forward simulations

In [ ]:
%%octave

clc

% Select src/det optodes to visualize
idxsrc = 2 # @param [1,2,3,4,5,6,7,8] {type:"raw"}
detid  = 3 # @param [1,2,3,4,5,6,7,8] {type:"raw"}
idxdet = detid + 8;

if ~isreal(phi0) % If FD simulation
  nplot = 2;
else
  nplot = 1;
end

figure,
subplot(nplot,2,1),plotmesh([cfg0.node log(abs(phi0(:,idxsrc)))],cfg0.elem,char(strcat('y>25'))),shading interp,colorbar,title('Src Log Amplitude') % Visualize fluence amplitude from det#10
subplot(nplot,2,2),plotmesh([cfg0.node log(abs(phi0(:,idxdet)))],cfg0.elem,char(strcat('y>25'))),shading interp,colorbar,title('Det Log Amplitude') % Visualize fluence amplitude from det#10
if ~isreal(phi0)
  subplot(nplot,2,3),plotmesh([cfg0.node   angle(phi0(:,idxsrc))],cfg0.elem,char(strcat('y>25'))),shading interp,colorbar,title('Src Phase') % Visualize fluence phase from det#10
  subplot(nplot,2,4),plotmesh([cfg0.node   angle(phi0(:,idxdet))],cfg0.elem,char(strcat('y>25'))),shading interp,colorbar,title('Det Phase') % Visualize fluence phase from det#10
end


% Visualize sensitivity matrix
figure,plotmesh([cfg0.node   log(abs(prod(phi0(:,[idxsrc idxdet]),2)))],cfg0.elem,'y>25') ,shading interp,colorbar,title('Log Amplitude Sensitivity Matrix') % Visualize sensitivity matrix src#1 & det#10
if ~isreal(phi0)
  figure,plotmesh([cfg0.node angle(prod(phi0(:,[idxsrc idxdet]),2))],cfg0.elem,'y>25') ,shading interp,colorbar,title('Phase Sensitivity Matrix') % Visualize sensitivity matrix src#1 & det#10
end

## Single-spectra Redbird-m reconstruction using point-source arrays of CW/FD illumination

In this section, we prepare a heterogenous mesh domain of 100x0100x40 mm with two inclusions with 2x absorption contrast located off-center in the z-axis (z = 15 mm).

First, we define the general setting to create the heterogenous domain along with a 4x4 point-source/detector array in a transmission geometry. We run a forward simulation on the heterogeneous domain (cfg0) to create simulated data (detphi0). Then, we create a homogeneous forward mesh (cfg) over which the inverse problem solution will be projected. We also create a sparse reconstruction mesh (recon) to speed up the Jacobian inversion and project those solutions over the forward mesh.

### General parameters for producing synthetic data

In this section we define the general parameters for running Redbird-m forward simulation to create synthetic data of a dual-inclusion heterogeneous domain using a 4x4 point-source array at the bottom of the domain and a similar one at the top of the domain for detection. We then create the forward and reconstruction meshes.

If generating synthetic data using FD point-source, we then perform a bulk optical properties fitting.

Last, using the simulated data (detphi0) and the forward and recon meshes, we perform an iterative reconstruction.

In [ ]:
%%octave

clear cfg0 cfg recon
clc

domsz = [100 100 40];     % Domain size
s0    = [35, 65, 15 5;... % Central position xyz and radius of the inclusion (in mm)
         60, 30, 15 5];

% Bounding box w/ 2 inclusions
resmesh = 5;
[nobbx,elembbx] = meshgrid5(0:resmesh:domsz(1),0:resmesh:domsz(2),0:resmesh:domsz(3)); % Meshbox domain (in mm)
fcbbx   = volface(elembbx);
[nobbx,fcbbx] = removeisolatednode(nobbx,fcbbx);
[nosp1,fcsp1] = meshasphere(s0(1,1:3),s0(1,4),1); % Sphare mesh location (s0, radious, maxvol)
[nosp2,fcsp2] = meshasphere(s0(2,1:3),s0(2,4),1); % Sphare mesh location (s1, radious, maxvol)
[no,fc]       = mergemesh(nobbx,fcbbx,nosp1,fcsp1,nosp2,fcsp2); % Merge meshbox and sphere meshes (FOR DUAL INCLUSION DOMAIN)
[cfg0.node, cfg0.elem] = s2m(no,fc(:,1:3),1,25,'tetgen1.5',[11 1 1;s0(1,1:3);s0(2,1:3)]); % Generate measured/real heterogeneous domain
clear nobbx elembbx fcbbx nosp1 fcsp1 nosp2 fcsp2 no fc

% Visualize domain
figure, plotmesh(cfg0.node, cfg0.elem,char(strcat('z<',num2str(s0(1,3)))))

% General settings for RB simulations
cfg0.prop = [     0  0 1 1   ; ...
             0.006 0.8 0 1.37; ...
             0.012 0.8 0 1.37; ...
             0.012 0.8 0 1.37];
mfreq      = 135 # @param [0, 135] {type:"raw"}
cfg0.omega = mfreq*1e6*2*pi;

% Define parameters for RB point-source array simulation
nsrc        = [4 4]; % Number of point src/det in xy axis
[xi,yi]     = meshgrid(linspace(20,80,nsrc(1)),linspace(20,80,nsrc(2)));
cfg0.srcpos = [xi(:),yi(:),zeros(numel(yi),1)];
cfg0.detpos = [xi(:),yi(:),domsz(3)*ones(numel(yi),1)];
cfg0.srcdir = [0 0  1];
cfg0.detdir = [0 0 -1];

cfg = cfg0;
[cfg0,sd0] = rbmeshprep(cfg0);

% Run RB forward simulation
[detphi0,phi0] = rbrunforward(cfg0,'sd',sd0);

% Set fwd mesh to be homogeneous
cfg = rbsetmesh(cfg,cfg.node,cfg.elem,cfg.prop,ones(size(cfg.node,1),1));

% Prepare recon mesh
resmesh    = 5;
[no,elem]  = meshgrid5(0:resmesh:domsz(1),0:resmesh:domsz(2),0:resmesh:domsz(3)); % Meshbox domain (in mm)
fc         = volface(elem);
[no,fc]    = removeisolatednode(no,fc);
[recon.node, recon.elem] = s2m(no,fc(:,1:3),1,50,'tetgen1.5');
recon.elem = recon.elem(:,1:4);
[recon.mapid,recon.mapweight] = tsearchn(recon.node,recon.elem(:,1:4),cfg.node);
clear no elem fc

% Visualize simulated data
figure,plot(abs(detphi0(:))),title('Channel-wise amplitude')
if ~isreal(phi0)
  figure,plot(angle(detphi0(:))),title('Channel-wise phase')
end


### Run bulk optical properties fitting

If utilizing FD sources, we can then perform a bulk optical properties fitting to decouple absorption and reduce scatterring coefficients. Otherwise, set them to be the true background optical properties used to generate synthetic data.

The bulk optical properties are then set as initial guess for launching the iterative reconstruction procedure.

In [ ]:
%%octave

if cfg0.omega > 0 % If simulating FD data, run bulk fitting

  % Set initial guess
  recon.prop = [0     0   1 1    ; ...
                0.008 0.9 0 1.37];

  sd = rbsdmap(cfg);
  [newrecon,resid] = rbrun(cfg,recon,detphi0,sd,'mode','bulk','lambda',1e-5,'maxiter',5);

else % If running CW data, assing bulk optical properties

  % Bulk optical properties to be used
  newrecon.prop = [0       0 1 1    ; ...
                   0.006 0.8 0 1.37];

end

% Set nodal-based optical properties to the bulk fitted values
recon.prop = newrecon.prop(ones(size(recon.node,1),1)+1,:);
cfg.prop   = newrecon.prop(ones(size(cfg.node,1),1)+1,:);
cfg        = rmfield(cfg,'seg');
newrecon.prop

### Run iterative reconstruction

In [ ]:
%%octave

sd = rbsdmap(cfg);
[reconMS,residMS,cfgMS] = rbrun(cfg,recon,detphi0,sd,'mode','image','lambda',1e-6,'maxiter',5);

### Visualize reconstructed tomographic images

In [ ]:
%%octave

% Generate a cross section slice of the domain
[xi,yi]             = meshgrid(min(cfg.node(:,1),[],1):1:max(cfg.node(:,1),[],1),min(cfg.node(:,2),[],1):1:max(cfg.node(:,2),[],1));
[cutpos,cutvalue,~] = qmeshcut(reconMS.elem,reconMS.node,(reconMS.prop(:,1)),char(strcat('z =',num2str(s0(1,3)))));
clear vphi
vphi(:,:,1)         = griddata(cutpos(:,1),cutpos(:,2),cutvalue,xi,yi);

% Visualize results

figure,
subplot(1,2,1);
axis equal
hold on;
imagesc(vphi);
colorbar;
axis tight;
for mm = 1:size(s0,1)
    hold on,plot(1:size(vphi,2),s0(mm,2).*ones(1,size(vphi,2)),'LineWidth',4,'LineStyle','-.')
end
subplot(1,2,2)
for mm = 1:size(s0,1)
    hold on,plot(vphi(s0(mm,2),:))
end

## Single-spectra Redbird-m reconstruction using wide-field source/detection of CW/FD illumination

In this section, we prepare a heterogenous mesh domain of 100x0100x40 mm with two inclusions with 2x absorption contrast located off-center in the z-axis (z = 15 mm).

First, we define the general setting to create the heterogenous domain along with a total of 32 wide-field source patterns in a transmission geometry. We run a forward simulation on the heterogeneous domain (cfg0) to create simulated data (detphi0). Then, we create a homogeneous forward mesh (cfg) over which the inverse problem solution will be projected. We also create a sparse reconstruction mesh (recon) to speed up the Jacobian inversion and project those solutions over the forward mesh.

### General parameters for producing synthetic data

In this section we define the general parameters for running Redbird-m forward simulation to create synthetic data of a dual-inclusion heterogeneous domain using a total of 32 wide-field source patterns (16 in the x-direction and 16 in the y-direction) located at the bottom of the domain and a similar one at the top of the domain for detection. We then create the forward and reconstruction meshes.

If generating synthetic data using FD point-source, we then perform a bulk optical properties fitting.

Last, using the simulated data (detphi0) and the forward and recon meshes, we perform an iterative reconstruction.

In [ ]:
%%octave

clear cfg0 cfg recon
clc

domsz = [100 100 40];     % Domain size
s0    = [35, 65, 15 5;... % Central position xyz and radius of the inclusion (in mm)
         60, 30, 15 5];

% Bounding box w/ 2 inclusions
resmesh = 5;
[nobbx,elembbx] = meshgrid5(0:resmesh:domsz(1),0:resmesh:domsz(2),0:resmesh:domsz(3)); % Meshbox domain (in mm)
fcbbx   = volface(elembbx);
[nobbx,fcbbx] = removeisolatednode(nobbx,fcbbx);
[nosp1,fcsp1] = meshasphere(s0(1,1:3),s0(1,4),1); % Sphare mesh location (s0, radious, maxvol)
[nosp2,fcsp2] = meshasphere(s0(2,1:3),s0(2,4),1); % Sphare mesh location (s1, radious, maxvol)
[no,fc]       = mergemesh(nobbx,fcbbx,nosp1,fcsp1,nosp2,fcsp2); % Merge meshbox and sphere meshes (FOR DUAL INCLUSION DOMAIN)
[cfg0.node, cfg0.elem] = s2m(no,fc(:,1:3),1,25,'tetgen1.5',[11 1 1;s0(1,1:3);s0(2,1:3)]); % Generate measured/real heterogeneous domain
clear nobbx elembbx fcbbx nosp1 fcsp1 nosp2 fcsp2 no fc

% Visualize domain
figure, plotmesh(cfg0.node, cfg0.elem,char(strcat('z<',num2str(s0(1,3)))))

% General settings for RB simulations
cfg0.prop = [     0  0 1 1   ; ...
             0.006 0.8 0 1.37; ...
             0.012 0.8 0 1.37; ...
             0.012 0.8 0 1.37];
mfreq      = 135 # @param [0, 135] {type:"raw"}
cfg0.omega = mfreq*1e6*2*pi;

% Define parameter for RB widefield simulation
nBarpats   = 16; % Number of bar patterns in x/y directions
srcpattern = diag(ones(1,nBarpats));
srcpattern = permute(repmat(srcpattern,[1,1,size(srcpattern, 1)]),[2 3 1]);
srcpattern = cat(3,srcpattern,permute(srcpattern,[2 1 3]));
detpattern = srcpattern;
cfg0.srctype    = 'pattern';
cfg0.srcpos     = [20 20  0];
cfg0.srcdir     = [ 0  0  1];
cfg0.srcparam1  = [60  0 0 0];
cfg0.srcparam2  = [ 0 60 0 0];
cfg0.srcpattern = srcpattern;
cfg0.srcweight  = ones(1,nBarpats*2);
cfg0.dettype    = 'pattern';
cfg0.detpos     = [20 20 domsz(3)];
cfg0.detdir     = [ 0  0 -1];
cfg0.detparam1  = [60  0  0 0];
cfg0.detparam2  = [ 0 60  0 0];
cfg0.detpattern = srcpattern;
cfg0.detweight  = ones(1,nBarpats*2);

cfg = cfg0;
[cfg0,sd0] = rbmeshprep(cfg0);

% Run RB forward simulation
[detphi0,phi0] = rbrunforward(cfg0,'sd',sd0);

% Set fwd mesh to be homogeneous
cfg = rbsetmesh(cfg,cfg.node,cfg.elem,cfg.prop,ones(size(cfg.node,1),1));

% Prepare recon mesh
resmesh    = 5;
[no,elem]  = meshgrid5(0:resmesh:domsz(1),0:resmesh:domsz(2),0:resmesh:domsz(3)); % Meshbox domain (in mm)
fc         = volface(elem);
[no,fc]    = removeisolatednode(no,fc);
[recon.node, recon.elem] = s2m(no,fc(:,1:3),1,50,'tetgen1.5');
recon.elem = recon.elem(:,1:4);
[recon.mapid,recon.mapweight] = tsearchn(recon.node,recon.elem(:,1:4),cfg.node);
clear no elem fc

% Visualize simulated data
figure,plot(abs(detphi0(:))),title('Single=pixel amplitude')
if ~isreal(phi0)
  figure,plot(angle(detphi0(:))),title('Single-pixel phase')
end


### Run bulk optical properties fitting

If utilizing FD sources, we can then perform a bulk optical properties fitting to decouple absorption and reduce scatterring coefficients. Otherwise, set them to be the true background optical properties used to generate synthetic data.

The bulk optical properties are then set as initial guess for launching the iterative reconstruction procedure.

In [ ]:
%%octave

if cfg0.omega > 0 % If simulating FD data, run bulk fitting

  % Set initial guess
  recon.prop = [0     0   1 1    ; ...
                0.008 0.9 0 1.37];

  sd = rbsdmap(cfg);
  [newrecon,resid] = rbrun(cfg,recon,detphi0,sd,'mode','bulk','lambda',1e-5,'maxiter',3);

else % If running CW data, assing bulk optical properties

  % Bulk optical properties to be used
  newrecon.prop = [0       0 1 1    ; ...
                   0.006 0.8 0 1.37];

end

% Set nodal-based optical properties to the bulk fitted values
recon.prop = newrecon.prop(ones(size(recon.node,1),1)+1,:);
cfg.prop   = newrecon.prop(ones(size(cfg.node,1),1)+1,:);
cfg        = rmfield(cfg,'seg');
newrecon.prop

### Run iterative reconstruction

In [ ]:
%%octave

sd = rbsdmap(cfg);
[reconMS,residMS,cfgMS] = rbrun(cfg,recon,detphi0,sd,'mode','image','lambda',1e-6,'maxiter',3);

### Visualize reconstructed tomographic images

In [ ]:
%%octave

% Generate a cross section slice of the domain
[xi,yi]             = meshgrid(min(cfg.node(:,1),[],1):1:max(cfg.node(:,1),[],1),min(cfg.node(:,2),[],1):1:max(cfg.node(:,2),[],1));
[cutpos,cutvalue,~] = qmeshcut(reconMS.elem,reconMS.node,(reconMS.prop(:,1)),char(strcat('z =',num2str(s0(1,3)))));
clear vphi
vphi(:,:,1)         = griddata(cutpos(:,1),cutpos(:,2),cutvalue,xi,yi);

% Visualize results
figure,
subplot(1,2,1);
axis equal
hold on;
imagesc(vphi);
colorbar;
axis tight;
for mm = 1:size(s0,1)
    hold on,plot(1:size(vphi,2),s0(mm,2).*ones(1,size(vphi,2)),'LineWidth',4,'LineStyle','-.')
end
subplot(1,2,2)
for mm = 1:size(s0,1)
    hold on,plot(vphi(s0(mm,2),:))
end